# Greenblatt Magic formula

## The Magic Formula: Explanation and Formula

The **Magic Formula** is an investment strategy developed by **Joel Greenblatt** to identify high-quality companies that are also undervalued. It ranks companies based on two key financial ratios:

---

## 📌 Core Idea

> **Buy good companies at cheap prices.**

The strategy identifies:
- **"Good companies"** → those with high returns on capital (efficient use of capital).
- **"Cheap companies"** → those with high earnings yield (undervalued based on operating profits).

---

## 🧮 The Formula

1. **Earnings Yield (EY):**
$$
\text{Earnings Yield} = \frac{\text{EBIT}}{\text{Enterprise Value}}
$$
- Measures how cheap the stock is.
- EBIT = Earnings Before Interest and Taxes
- Enterprise Value = Market Cap + Debt - Cash

2. **Return on Capital (ROC):**
$$
\text{Return on Capital} = \frac{\text{EBIT}}{\text{Net Working Capital} + \text{Net Fixed Assets}}
$$
- Measures the quality of the business (how efficiently it uses its capital).

---

## 🔍 Implementation Steps

1. **Filter the universe**: Remove financials, utilities, and companies with very small market cap.
2. **Rank all remaining stocks** by:
   - Earnings Yield (high = better)
   - Return on Capital (high = better)
3. **Compute combined rank**:
   $$
   \text{Combined Rank} = \text{Rank}_{EY} + \text{Rank}_{ROC}
   $$
4. **Sort by Combined Rank** (lowest = best overall).
5. **Pick top N stocks** (e.g., top 20–30).
6. **Hold for 1 year**, rebalance annually.

---

## ✅ Why It Works

- Avoids paying too much for popular stocks.
- Focuses on operationally efficient, consistently profitable companies.
- Enforces a disciplined, rules-based approach.

---

## ⚠️ Notes and Caveats

- Avoids subjective judgement; however, **screening accuracy** depends on **quality of financial data**.
- May underperform in short-term or irrational markets.
- Works best over a multi-year horizon (3–5+ years).
- I have added a minimum 40% margin restriction to the original formula. 

---


In [1]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
import requests
import pandas as pd
from tqdm import tqdm

api_key = os.getenv('financial_modeling_prep_api_key')
assert api_key is not None

In [2]:


# Retry settings: 5 attempts with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=10, max=60))
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_magic_formula(api_key, symbols, minimum_margin=0.4, minimum_marketcap=500000000):
    base_url = "https://financialmodelingprep.com/stable"

    results = []

    for symbol in tqdm(symbols, desc="Processing symbols"):
        try:
            # check market cap
            profile_url = f"{base_url}/enterprise-values?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            if not profile_data or profile_data[0].get('marketCapitalization') < minimum_marketcap:
                continue

            # Check sector
            profile_url = f"{base_url}/search-exchange-variants?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            if not profile_data or profile_data[0].get('sector') in ['Financial Services', "Real Estate"]:
                continue

            # Step 2: Fetch income statement
            income_url = f"{base_url}/income-statement/?symbol={symbol}&apikey={api_key}"
            income_data = fetch_json(income_url)[0]

            # Step 3: Fetch balance sheet
            balance_url = f"{base_url}/balance-sheet-statement/?symbol={symbol}&apikey={api_key}"
            balance_data = fetch_json(balance_url)[0]

            # Step 4: Fetch enterprise value
            ev_url = f"{base_url}/enterprise-values/?symbol={symbol}&apikey={api_key}"
            ev_data = fetch_json(ev_url)[0]

            # Extract needed values
            ebit = income_data.get('ebit')
            revenue = income_data.get('revenue')
            total_assets = balance_data.get('totalAssets')
            current_liabilities = balance_data.get('totalCurrentLiabilities')
            enterprise_value = ev_data.get('enterpriseValue')

            net_income = income_data.get('netIncome')
            if net_income is None or net_income <= 0:
                continue

            # Check for missing data
            if None in (ebit, revenue, total_assets, current_liabilities, enterprise_value):
                print(f"missing data for {symbol}")
                continue

            # Apply EBIT margin constraint
            ebit_margin = ebit / revenue
            if ebit_margin < minimum_margin:
                continue

            # Calculate metrics
            earnings_yield = ebit / enterprise_value
            capital = total_assets - current_liabilities
            if capital <= 0:
                continue
            return_on_capital = ebit / capital

            results.append({
                'symbol': symbol,
                'ebit_margin': ebit_margin,
                'earnings_yield': earnings_yield,
                'return_on_capital': return_on_capital
            })

        except Exception as e:
            print(f'Symbol: {symbol}\n')
            print(e)

    # Step 5: Create DataFrame and rank
    df = pd.DataFrame(results)
    if df.empty:
        return df

    df['ey_rank'] = df['earnings_yield'].rank(ascending=False)
    df['roc_rank'] = df['return_on_capital'].rank(ascending=False)
    df['combined_rank'] = df['ey_rank'] + df['roc_rank']
    df_sorted = df.sort_values(by='combined_rank')

    return df_sorted[['symbol', 'ebit_margin', 'earnings_yield', 'return_on_capital', 'combined_rank']]

In [ ]:

symbols = pd.read_csv("russel1000.csv")
symbols = symbols['Ticker'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.15)
print(top_stocks)
russel1000_filtered = top_stocks

Processing symbols:   1%|▋                                                            | 11/1012 [00:05<06:22,  2.62it/s]

Symbol: BRKB

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols:  13%|████████                                                    | 135/1012 [01:28<04:16,  3.42it/s]

In [ ]:
symbols = pd.read_csv("all_listed_companies.csv")
symbols = symbols['Symbol'].to_list()

top_stocks = get_magic_formula(api_key, symbols)
print(top_stocks)
world_filtered = top_stocks

## Tests

In [58]:
base_url = "https://financialmodelingprep.com/stable"
api_key = 'mRup6c6gbK2tGlpqfxoDmrm9SyEsMEjC'
profile_url = f"{base_url}/enterprise-values?symbol=JPM&apikey={api_key}"
response = requests.get(profile_url)
response.raise_for_status()
profile_data = response.json()

In [62]:
profile_data[0]

{'symbol': 'JPM',
 'date': '2024-12-31',
 'stockPrice': 239.71,
 'numberOfShares': 2873900000,
 'marketCapitalization': 688902569000,
 'minusCashAndCashEquivalents': 469317000000,
 'addTotalDebt': 454311000000,
 'enterpriseValue': 673896569000}

In [ ]:
top_stocks